# 🚀 Install, Import, and Log in

### Step 0️⃣: Install W&B

In [5]:
%%capture
#!pip install wandb

### Step 1️⃣: Import W&B and Login

In [6]:
import tensorflow as tf
from tensorflow import keras

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.feather as feather
from sklearn.model_selection import train_test_split 
from tensorflow.keras import regularizers
import math

%matplotlib inline

In [7]:
import wandb
from wandb.keras import WandbCallback


!wandb.login()

/bin/bash: -c: line 1: syntax error: unexpected end of file


Read data in

In [8]:
fluxData_df = feather.read_feather('data/fluxData.feather')
zernikeData_df = feather.read_feather('data/zernikeData.feather')
#fluxData_df


# 👩‍🍳 Prepare Dataset

In [9]:
# Prepare the training dataset
X = fluxData_df
y = zernikeData_df
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)


## Config parameter

# Create Model

In [10]:
def Model():
    inputs = keras.Input(shape=(19,), name="digits")
    x1 = keras.layers.Dense(wandb.config.layer1, activation="relu", name = "conv1D_1")(inputs)
    x2 = keras.layers.Dense(wandb.config.layer2, activation="relu")(x1)
    x3 = keras.layers.Dense(wandb.config.layer3, activation="relu")(x2)
    outputs = keras.layers.Dense(9, name="predictions")(x3)

    return keras.Model(inputs=inputs, outputs=outputs)
    
#AoSweepModel = Model()    
#AoSweepModel.summary()

In [11]:
#keras.utils.plot_model(AoSweepModel, show_shapes=True)

## Reshape training data

In [12]:
input_train = x_train.to_numpy()[:,:,np.newaxis]
input_test = x_test.to_numpy()[:,:,np.newaxis]

Train simple model

In [13]:
def training():
    run = wandb.init(project="Ao_Paper_Model",
    notes="Learning rate scheduler",
    tags=['Ao', "Paper", "lr_schedule" ,"1DConv"])
    AoSweepModel = Model() 
    opt = keras.optimizers.Adam(learning_rate= wandb.config.learning_rate)
    AoSweepModel.compile(optimizer= opt,  loss="mse", metrics=[keras.metrics.RootMeanSquaredError()])
    AoSweepModel.fit(input_train, y_train, batch_size= 128, epochs= wandb.config.epochs, validation_split=0.1, callbacks=[WandbCallback()])
    callbacks=[WandbCallback()]
#training()

## Define sweep

In [14]:
os.environ["WANDB_NOTEBOOK_NAME"] = "AoPaperModelSweep.ipynb"
sweep_config = {
  "name" : "PaperVersionSweep",
  "method" : "random",
  'metric': {
       'name': 'accuracy',
       'goal': 'maximize',
    },
  "parameters" : {
    "epochs" : {
      "values" : [20, 40]
    },
    "layer1" : {
      "values" : [4000, 2000, 500]
    },
    "layer2" : {
      "values" : [800, 300]
    },
    "layer3" : {
      "values" : [100, 50]
    },
    "learning_rate" :{
      "min": 1e-4,
      "max": 5e-3
    }
  }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: emhmlz86
Sweep URL: https://wandb.ai/uwe-sterr/uncategorized/sweeps/emhmlz86


In [15]:
count = 100 # number of runs to execute
wandb.agent(sweep_id, function=training, count=count)

wandb: Agent Starting Run: s96hrgw8 with config:
wandb: 	epochs: 40
wandb: 	layer1: 4000
wandb: 	layer2: 800
wandb: 	layer3: 100
wandb: 	learning_rate: 0.00037842463324696456
wandb: Currently logged in as: uwe-sterr (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/40
301/372 [=======================>......] - ETA: 1s - loss: 0.0097 - root_mean_squared_error: 0.0983

wandb: Ctrl + C detected. Stopping sweep.
